In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

In [ ]:
data = pd.read_csv("dataset.csv")
data.head()

In [ ]:
# first two columns are unnecessary (index)
data.drop(data.columns[[0, 1]], axis = 1, inplace = True)
data.head()

In [ ]:
labels = ["radiation", "ozone", "temperature", "wind"]
fig, axs = plt.subplots(4, 4, constrained_layout=True)
fig.set_size_inches(7, 7)
for i in range(4):
    for j in range(4):
        axs[i, j].axes.xaxis.set_visible(False)
        axs[i, j].axes.yaxis.set_visible(False)
        if i == j:
            axs[i, j].plot()
            axs[i, j].text(.5, .5, labels[i].capitalize(),  horizontalalignment='center', verticalalignment='center', transform=axs[i, j].transAxes, fontsize=13)
        else:
            if i == 0 or i == 3:
                axs[i, j].axes.xaxis.set_visible(True)
            if i == 0:
                axs[i, j].xaxis.tick_top()
            if j == 0 or j == 3:
                axs[i, j].axes.yaxis.set_visible(True)
            if j == 3:
                axs[i, j].yaxis.tick_right()
            axs[i, j].scatter(data[labels[j]], data[labels[i]], s=15, color="black")

In [ ]:
def tricubic(x):
        y = np.zeros_like(x)
        idx = (x >= -1) & (x <= 1)
        y[idx] = np.power(1.0 - np.power(np.abs(x[idx]), 3), 3)
        return y
plt.plot(np.linspace(-2, 2, 100), tricubic(np.linspace(-2, 2, 100)))

In [ ]:
class Loess:
    @staticmethod
    def standarize_data(data):
        data_mean = data.mean(0)
        data_std = data.std(0)
        return (data - data_mean) / data_std, data_mean, data_std

    def __init__(self, xx, yy, degree):
        xx = np.array(xx)
        yy = np.array(yy)
        self.s_xx, self.mean_xx, self.std_xx = self.standarize_data(xx)
        self.s_yy, self.mean_yy, self.std_yy = self.standarize_data(yy)
        self.degree = degree

    def set_degree(self, degree):
        self.degree = degree

    @staticmethod
    def get_weights(distances, min_range):
        max_distance = max(distances[min_range])
        weights = tricubic(distances[min_range] / max_distance)
        return weights

    def standarize_x(self, value):
        return (value - self.mean_xx) / self.std_xx

    def destandarize_y(self, value):
        return value * self.std_yy + self.mean_yy

    def estimate(self, x, f):
        q = int(len(self.s_xx) * f)

        s_x = self.standarize_x(x)
        distances = np.linalg.norm(self.s_xx - s_x, axis=1)
        min_idx = np.argsort(distances)[:q]

        weights = self.get_weights(distances, min_idx)
        wm = np.diag(weights)
        
        xm = self.s_xx[min_idx]
        ym = self.s_yy[min_idx]

        poly = PolynomialFeatures(degree=self.degree, include_bias=False)
        xm = poly.fit_transform(xm)
        xp = poly.fit_transform(s_x.reshape(1, -1))[0]

        xmt_wm = xm.T @ wm
        beta = np.linalg.pinv(xmt_wm @ xm) @ xmt_wm @ ym
        y = beta.T @ xp
        return self.destandarize_y(y)[0]

In [ ]:
X = data[["radiation", "temperature", "wind"]]
Y = data[["ozone"]]
loess = Loess(X, Y, 1)
estimations = []
for i in range(len(X)):
    x = np.array(X.iloc[[i]])[0]
    estimations.append(loess.estimate(x, 0.4))
data["linear_0.4_estimation"] = estimations
data["linear_0.4_residual"] = data["ozone"] - data["linear_0.4_estimation"]

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
res = stats.probplot(np.sort(data["linear_0.4_residual"]), plot=plt, fit=False)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["linear_0.4_estimation"], abs(data["linear_0.4_residual"]), facecolors='none', edgecolors='black', s=20)
plt.xlabel("Fitted Values")
plt.ylabel("Absolute Residuals")

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["radiation"], data["linear_0.4_residual"], facecolors='none', edgecolors='black', s=20)
plt.xlabel("Solar Radiation")
plt.ylabel("Residuals")

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["temperature"], data["linear_0.4_residual"], facecolors='none', edgecolors='black', s=20)
plt.xlabel("Temperature")
plt.ylabel("Residuals")

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["wind"], data["linear_0.4_residual"], facecolors='none', edgecolors='black', s=20)
plt.xlabel("Wind Speed")
plt.ylabel("Residuals")

In [ ]:
Y_cube_root = np.power(data[["ozone"]], 1/3)
loess = Loess(X, Y_cube_root, 2)

In [ ]:
# fixed wind speed and radiation
wind_speed = [5, 10, 15]
temperature = np.linspace(60, 95, 100)
radiation = [290, 170, 50]

fig, axs = plt.subplots(3, 3, constrained_layout=True)
fig.set_size_inches(7, 7)

estimations = []
for i, wind in enumerate(wind_speed):
    for j, rad in enumerate(radiation):
        estimations = []
        for temp in temperature:
            x = np.array([rad, temp, wind])
            estimations.append(loess.estimate(x, 0.8))
        if j != 0:
            axs[i, j].axes.yaxis.set_visible(False)
        if i != 2:
            axs[i, j].axes.xaxis.set_visible(False)
        axs[j, i].set_ylim(0, 7)
        axs[j, i].plot(temperature, estimations, color="black")

In [ ]:
# fixed wind speed and temperature
wind_speed = [5, 10, 15]
temperature = [90, 76, 62]
radiation = np.linspace(0, 300, 100)

fig, axs = plt.subplots(3, 3, constrained_layout=True)
fig.set_size_inches(7, 7)

estimations = []
for i, wind in enumerate(wind_speed):
    for j, temp in enumerate(temperature):
        estimations = []
        for rad in radiation:
            x = np.array([rad, temp, wind])
            estimations.append(loess.estimate(x, 0.8))
        if j != 0:
            axs[i, j].axes.yaxis.set_visible(False)
        if i != 2:
            axs[i, j].axes.xaxis.set_visible(False)
        axs[j, i].set_ylim(0, 7)
        axs[j, i].plot(radiation, estimations, color="black")

In [ ]:
# fixed temperature and radiation
wind_speed = np.linspace(3, 20, 100)
temperature = [62, 76, 90]
radiation = [290, 170, 50]

fig, axs = plt.subplots(3, 3, constrained_layout=True)
fig.set_size_inches(7, 7)

estimations = []
for i, temp in enumerate(temperature):
    for j, rad in enumerate(radiation):
        estimations = []
        for wind in wind_speed:
            x = np.array([rad, temp, wind])
            estimations.append(loess.estimate(x, 0.8))
        if j != 0:
            axs[i, j].axes.yaxis.set_visible(False)
        if i != 2:
            axs[i, j].axes.xaxis.set_visible(False)
        axs[j, i].set_ylim(0, 7)
        axs[j, i].plot(wind_speed, estimations, color="black")